In [1]:
!pip  install pandas numpy scikit-learn matplotlib seaborn

  Using cached numpy-2.3.4-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.4-cp313-cp313-win_amd64.whl (12.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.3.4 which is incompatible.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import io
import joblib

In [4]:
import pandas as pd
df = pd.read_csv('data/disease_train_data.csv')
print(f"DataFrame loaded with {len(df)} rows.")

DataFrame loaded with 55 rows.


In [5]:
def clean_col_name(col):
    
    col = col.strip().lower()
    col = col.replace(" ", "_")
    col = col.replace("-", "_")
    col = col.replace("(", "")
    col = col.replace(")", "")
    col = col.replace(".", "")
    col = "_".join(filter(None, col.split("_")))
    return col
df.columns = [clean_col_name(col) for col in df.columns]

print("\n--- Dataset Overview ---")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print(f"First 5 columns: {df.columns[:5].tolist()}")
print(f"Last 5 columns: {df.columns[-5:].tolist()}")


--- Dataset Overview ---
Shape: (55, 147)
Columns: 147
First 5 columns: ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering']
Last 5 columns: ['bone_pain', 'tingling_in_hands_feets', 'memory_loss', 'blood_deficiency', 'prognosis']


In [6]:
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,dry_skin,bleeding_gums_swollen_gums,nosebleeds,slow_wound_healing,�tooth_loss�,bone_pain,tingling_in_hands_feets,memory_loss,blood_deficiency,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [7]:
df.tail()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,dry_skin,bleeding_gums_swollen_gums,nosebleeds,slow_wound_healing,�tooth_loss�,bone_pain,tingling_in_hands_feets,memory_loss,blood_deficiency,prognosis
50,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Calcium Deficiency
51,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,Vitamin A Deficiency
52,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,Vitamin C Deficiency
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Vitamin D Deficiency
54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,Vitamin B12 Deficiency


In [8]:
df.shape

(55, 147)

In [9]:

# Apply column cleaning to the entire DataFrame
df.columns = [clean_col_name(col) for col in df.columns]
df = df.fillna(0)
    # --- EDA Step: Display Head and Tail of Cleaned Data ---
print("--- Head of Cleaned Dataset (First 5 Rows) ---")
print(df.head().to_markdown(index=False))

print("\n--- Tail of Cleaned Dataset (Last 5 Rows) ---")
print(df.tail().to_markdown(index=False))
print("-" * 40)


--- Head of Cleaned Dataset (First 5 Rows) ---
|   itching |   skin_rash |   nodal_skin_eruptions |   continuous_sneezing |   shivering |   chills |   joint_pain |   stomach_pain |   acidity |   ulcers_on_tongue |   muscle_wasting |   vomiting |   burning_micturition |   spotting_urination |   fatigue |   weight_gain |   anxiety |   cold_hands_and_feets |   mood_swings |   weight_loss |   restlessness |   lethargy |   patches_in_throat |   irregular_sugar_level |   cough |   high_fever |   sunken_eyes |   breathlessness |   sweating |   dehydration |   indigestion |   headache |   yellowish_skin |   dark_urine |   nausea |   loss_of_appetite |   pain_behind_the_eyes |   back_pain |   constipation |   abdominal_pain |   diarrhoea |   mild_fever |   yellow_urine |   yellowing_of_eyes |   acute_liver_failure |   fluid_overload |   swelling_of_stomach |   swelled_lymph_nodes |   malaise |   blurred_and_distorted_vision |   phlegm |   throat_irritation |   redness_of_eyes |   sinus_pressure

In [10]:
if 'prognosis' not in df.columns:
        print("Error: 'prognosis' column not found after cleaning. Check your dataset structure.")
        # Cannot proceed without target column
else:
        # Separate features (X) and target (y)
        X = df.drop('prognosis', axis=1)
        y = df['prognosis']
        X = X.loc[:, (X != 0).any(axis=0)]
        X = X.astype(np.int8)
        # --- NEW CONFIRMATION STEP: Check Disease Frequency ---
        print("--- Disease Frequency (Top 5) ---")
        print(y.value_counts().head(5).to_markdown(numalign="left", stralign="left"))
        print("-" * 40)

--- Disease Frequency (Top 5) ---
| prognosis           | count   |
|:--------------------|:--------|
| Fungal infection    | 2       |
| Allergy             | 1       |
| GERD                | 1       |
| Chronic cholestasis | 1       |
| Drug Reaction       | 1       |
----------------------------------------


In [11]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

        # Get the list of symptom columns for later use in prediction
symptom_columns = X.columns.tolist()


In [12]:
print("--- Available Symptoms in Dataset (Features) ---")
        # Find and print the specific symptom we are troubleshooting
try:
    spider_veins_col = [col for col in symptom_columns if 'spider' in col]
    print(f"Verified 'Spider veins on skin' column name: {spider_veins_col[0] if spider_veins_col else 'Not Found'}")
except:
    pass
print(f"Total number of samples: {len(X)}")
print(f"Number of symptom features: {len(symptom_columns)}")
print(f"Number of unique diseases: {len(le.classes_)}")
print("-" * 40)

--- Available Symptoms in Dataset (Features) ---
Verified 'Spider veins on skin' column name: spider_veins_on_skin
Total number of samples: 55
Number of symptom features: 145
Number of unique diseases: 54
----------------------------------------


In [43]:
X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.05, random_state=42 # CHANGED test_size to 0.1
        )

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print("-" * 40)

Training samples: 52
Testing samples: 3
----------------------------------------


In [44]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
MODEL_FILENAME = 'disease_model.pkl'
ENCODER_FILENAME = 'label_encoder.pkl'
try:
    joblib.dump(model, MODEL_FILENAME)
    joblib.dump(le, ENCODER_FILENAME)
    print(f"\nModel successfully saved to {MODEL_FILENAME}")
    print(f"Label Encoder successfully saved to {ENCODER_FILENAME}")
except Exception as e:
       print(f"Error saving model files: {e}")
print("-" * 40)


        # --- 5. Evaluate the Model ---
y_pred = model.predict(X_test)

        # --- Logic to safely generate Classification Report ---
        # Get the unique encoded labels present in the test set (y_test)
unique_labels_in_test = np.unique(y_test)
        # Get the corresponding disease names for those unique labels
target_names_filtered = le.inverse_transform(unique_labels_in_test)

print("--- Model Evaluation ---")
print("Classification Report (Filtered):")
        # Use the filtered target_names and labels in the report
report = classification_report(
            y_test, y_pred,
            labels=unique_labels_in_test, # Explicitly tell report which labels to look for
            target_names=target_names_filtered,
            zero_division=0
        )
print(report)

print("\nConfusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_pred))
print("-" * 40)



Model successfully saved to disease_model.pkl
Label Encoder successfully saved to label_encoder.pkl
----------------------------------------
--- Model Evaluation ---
Classification Report (Filtered):
                     precision    recall  f1-score   support

    Hyperthyroidism       0.00      0.00      0.00       1.0
     Hypothyroidism       0.00      0.00      0.00       1.0
Peptic ulcer diseae       0.00      0.00      0.00       1.0

          micro avg       0.00      0.00      0.00       3.0
          macro avg       0.00      0.00      0.00       3.0
       weighted avg       0.00      0.00      0.00       3.0


Confusion Matrix (Test Set):
[[0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]]
----------------------------------------


In [45]:
def predict_disease(symptoms_list):
            """
            Predicts the disease based on a list of reported symptoms.

            Args:
                symptoms_list (list): A list of symptom strings (e.g., ['itching', 'vomiting']).
                                    The symptom names must match the column names in the training data.
            """
            # Create an empty symptom array (all 0s)
            input_data = np.zeros(len(symptom_columns), dtype=int)

            # Create a Series for easy matching
            symptom_series = pd.Series(input_data, index=symptom_columns)

            # Map the reported symptoms to the correct index/column and set the value to 1
            valid_symptoms = []
            for symptom in symptoms_list:
                # Clean the input symptom name to match the column names
                cleaned_symptom = clean_col_name(symptom)
                if cleaned_symptom in symptom_columns:
                    symptom_series[cleaned_symptom] = 1
                    valid_symptoms.append(symptom)
                else:
                    # Note: The cleaned symptom for 'Blood in urine(hematuria)' is 'blood_in_urine_hematuria'
                    print(f"Warning: Symptom '{symptom}' (cleaned to '{cleaned_symptom}') is NOT a known feature in the model.")

            # Reshape the data for prediction (1 row, N columns)
            input_df = pd.DataFrame([symptom_series.values], columns=symptom_columns)

            # Make the prediction
            prediction_encoded = model.predict(input_df)

            # Decode the result back to the disease name
            predicted_disease = le.inverse_transform(prediction_encoded)[0]

            print("\n--- Prediction Result ---")
            print(f"Reported Symptoms: {', '.join(valid_symptoms) if valid_symptoms else 'None'}")
            print(f"Predicted Disease: {predicted_disease}")
            print("-------------------------")

            return predicted_disease

In [46]:
print("--- Example Predictions ---")
        # Example 1: A patient with itching and skin rash (consistent with Fungal infection)
predict_disease(['itching', 'skin rash'])

        # Example 2: A patient with stomach pain, vomiting, and headache
predict_disease(['stomach pain', 'vomiting', 'headache'])

        # Example 3: Test the specific symptom: 'Spider veins on skin' (pass in the raw string)
        # The output will confirm if the cleaning process successfully maps it to 'spider_veins_on_skin'
predict_disease(['high fever', 'Spider veins on skin', 'cough'])

        # NEW PREDICTION using the user's list
print("\n--- User-Requested Prediction ---")
predict_disease(['vomiting','fatigue','restlessness','nausea','back pain','Blood in urine(hematuria)'])

--- Example Predictions ---

--- Prediction Result ---
Reported Symptoms: itching, skin rash
Predicted Disease: Fungal infection
-------------------------

--- Prediction Result ---
Reported Symptoms: stomach pain, vomiting, headache
Predicted Disease: Paralysis (brain hemorrhage)
-------------------------

--- Prediction Result ---
Reported Symptoms: high fever, Spider veins on skin, cough
Predicted Disease: AIDS
-------------------------

--- User-Requested Prediction ---

--- Prediction Result ---
Reported Symptoms: vomiting, fatigue, restlessness, nausea, back pain
Predicted Disease: Kidney Stone
-------------------------


'Kidney Stone'

In [47]:
predict_disease(['loss of appetite','constipation','abdominal pain','diarrhoea','pain during bowel movements','bloody stool'])


--- Prediction Result ---
Reported Symptoms: loss of appetite, constipation, abdominal pain, diarrhoea, pain during bowel movements, bloody stool
Predicted Disease: Colon Cancer
-------------------------


'Colon Cancer'

In [48]:
predict_disease(['bloody stool','pain during bowel movements','loss of appetite','abdominal pain','constipation','constipation'])


--- Prediction Result ---
Reported Symptoms: bloody stool, pain during bowel movements, loss of appetite, abdominal pain, constipation, constipation
Predicted Disease: Colon Cancer
-------------------------


'Colon Cancer'

In [49]:
predict_disease(['bloody stool','pain during bowel movements'])


--- Prediction Result ---
Reported Symptoms: bloody stool, pain during bowel movements
Predicted Disease: Dimorphic hemmorhoids(piles)
-------------------------


'Dimorphic hemmorhoids(piles)'

In [50]:
predict_disease(['bloody stool'])


--- Prediction Result ---
Reported Symptoms: bloody stool
Predicted Disease: Dimorphic hemmorhoids(piles)
-------------------------


'Dimorphic hemmorhoids(piles)'

In [51]:
predict_disease(['night blindness'])


--- Prediction Result ---
Reported Symptoms: night blindness
Predicted Disease: Arthritis
-------------------------


'Arthritis'

In [52]:
predict_disease(['itching','spider veins on skin','fatigue','weight loss','yellowish skin','dark urine','nausea','loss of appetite','abdominal pain','yellowing of eyes'])


--- Prediction Result ---
Reported Symptoms: itching, spider veins on skin, fatigue, weight loss, yellowish skin, dark urine, nausea, loss of appetite, abdominal pain, yellowing of eyes
Predicted Disease: Liver Cirrhosis
-------------------------


'Liver Cirrhosis'

In [53]:
predict_disease(['Abdominal pain', 'diarrhea', 'nausea', 'vomiting'])


--- Prediction Result ---
Reported Symptoms: Abdominal pain, nausea, vomiting
Predicted Disease: Alcoholic hepatitis
-------------------------


'Alcoholic hepatitis'

In [54]:
predict_disease(['Chest pain', 'sweating', 'shortness of breath', 'nausea'])


--- Prediction Result ---
Reported Symptoms: Chest pain, sweating, nausea
Predicted Disease: Heart attack
-------------------------


'Heart attack'

In [55]:
predict_disease(['Persistent cough', 'weight loss', 'night sweats'])


--- Prediction Result ---
Reported Symptoms: weight loss
Predicted Disease: Arthritis
-------------------------


'Arthritis'

In [56]:
predict_disease(['Fever', 'cough', 'tiredness', 'loss of taste smell'])


--- Prediction Result ---
Reported Symptoms: cough
Predicted Disease: Arthritis
-------------------------


'Arthritis'

In [57]:
predict_disease(['Tingling hands/feet', 'tiredness', 'pale skin','anemia'])


--- Prediction Result ---
Reported Symptoms: None
Predicted Disease: Arthritis
-------------------------


'Arthritis'

In [58]:
predict_disease(['Bone pain', 'muscle weakness', 'fatigue'])


--- Prediction Result ---
Reported Symptoms: Bone pain, muscle weakness, fatigue
Predicted Disease: Vitamin D  Deficiency
-------------------------


'Vitamin D  Deficiency'

In [59]:
predict_disease(['Muscle cramps', 'brittle nails', 'bone pain'])


--- Prediction Result ---
Reported Symptoms: brittle nails, bone pain
Predicted Disease: Vitamin D  Deficiency
-------------------------


'Vitamin D  Deficiency'

In [60]:
predict_disease([ 'bleeding gums swollen gums','nosebleeds','dry skin'])


--- Prediction Result ---
Reported Symptoms: bleeding gums swollen gums, nosebleeds, dry skin
Predicted Disease: Vitamin C  Deficiency
-------------------------


'Vitamin C  Deficiency'

In [61]:
predict_disease(['Swelling in the limbs','Swollen lymph nodes','fever','weakness','chills','joint pain','fatigue'])


--- Prediction Result ---
Reported Symptoms: chills, joint pain, fatigue
Predicted Disease: Vitamin D  Deficiency
-------------------------


'Vitamin D  Deficiency'

In [62]:
predict_disease(['itching', 'skin rash', 'nodal skin eruptions', 'blurred and distorted vision'])


--- Prediction Result ---
Reported Symptoms: itching, skin rash, nodal skin eruptions, blurred and distorted vision
Predicted Disease: Fungal infection
-------------------------


'Fungal infection'

In [63]:
predict_disease(['stomach pain', 'acidity', 'ulcers on tongue', 'cough', 'vomiting', 'muscle weakness'])


--- Prediction Result ---
Reported Symptoms: stomach pain, acidity, ulcers on tongue, cough, vomiting, muscle weakness
Predicted Disease: GERD
-------------------------


'GERD'

In [64]:
predict_disease(['itching', 'vomiting', 'headache', 'yellowish skin', 'nausea', 'loss of appetite', 'diarrhea'])


--- Prediction Result ---
Reported Symptoms: itching, vomiting, headache, yellowish skin, nausea, loss of appetite
Predicted Disease: Chronic cholestasis
-------------------------


'Chronic cholestasis'

In [65]:
predict_disease(['muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts'])


--- Prediction Result ---
Reported Symptoms: muscle wasting, patches in throat, high fever, extra marital contacts
Predicted Disease: AIDS
-------------------------


'AIDS'

In [66]:
predict_disease(['skin rash', 'scurfing', 'silver like dusting', 'small dents in nails'])


--- Prediction Result ---
Reported Symptoms: skin rash, silver like dusting, small dents in nails
Predicted Disease: Psoriasis
-------------------------


'Psoriasis'

In [67]:
predict_disease(['fatigue', 'cold hands and feets', 'breathlessness', 'headache', 'dizziness', 'chest pain', 'palpitations', 'excessive hunger', 'family history', 'malaise'])


--- Prediction Result ---
Reported Symptoms: fatigue, cold hands and feets, breathlessness, headache, dizziness, chest pain, palpitations, excessive hunger, family history, malaise
Predicted Disease: Hypertension 
-------------------------


'Hypertension '

In [68]:
predict_disease(['fatigue', 'malaise', 'watering_from_eyes', 'bone_pain', 'tingling_in_hands_feets', 'memory_loss', 'blood_deficiency'])


--- Prediction Result ---
Reported Symptoms: fatigue, malaise, watering_from_eyes, bone_pain, tingling_in_hands_feets, memory_loss, blood_deficiency
Predicted Disease: Vitamin B12 Deficiency
-------------------------


'Vitamin B12 Deficiency'

In [69]:
predict_disease(['vomiting', 'fatigue', 'restlessness', 'lethargy', 'headache', 'nausea', 'dizziness', 'dehydration'])


--- Prediction Result ---
Reported Symptoms: vomiting, fatigue, restlessness, lethargy, headache, nausea, dizziness, dehydration
Predicted Disease: Sodium deficiency�
-------------------------


'Sodium deficiency�'

In [70]:
predict_disease(['joint pain', 'fatigue', 'abdominal pain', 'mild fever', 'malaise', 'phlegm', 'throat irritation', 'blurred vision', 'night blindness', 'dry eye'])


--- Prediction Result ---
Reported Symptoms: joint pain, fatigue, abdominal pain, mild fever, malaise, phlegm, throat irritation, blurred vision, night blindness, dry eye
Predicted Disease: Vitamin A  Deficiency
-------------------------


'Vitamin A  Deficiency'

In [71]:
predict_disease(['vomiting', 'fatigue', 'weight_loss', 'headache', 'nausea', 'loss of appetite', 'abdominal pain', 'diarrhea', 'mild fever', 'dizziness', 'dehydration'])


--- Prediction Result ---
Reported Symptoms: vomiting, fatigue, weight_loss, headache, nausea, loss of appetite, abdominal pain, mild fever, dizziness, dehydration
Predicted Disease: Food Poisoning
-------------------------


'Food Poisoning'

In [72]:
predict_disease(['vomiting', 'fatigue', 'anxiety', 'sweating', 'headache', 'nausea', 'irritability', 'depression', 'palpitations', 'excessive hunger'])


--- Prediction Result ---
Reported Symptoms: vomiting, fatigue, anxiety, sweating, headache, nausea, irritability, depression, palpitations, excessive hunger
Predicted Disease: Hypoglycemia
-------------------------


'Hypoglycemia'